Actividad 3.

1. Generar preguntas de interes o hipotesis de interes sobre el dataset elegido para el proyecto final.
2. Crear visualizaciones (univariados, bivariados o trivariados) junto con resumenes numericos basicos acordes con los tipos de variables disponibles.
3. Interpretar los resultados obtenidos.

In [1]:
import pandas as pd

Vacunas = pd.read_csv("Asignacion_vacuna_contra_COVID-19.csv")
Muertes = pd.read_csv("Fallecidos_COVID_en_Colombia.csv")

Preguntas 

1. Cuantas muertes por covid-19 se reportan por region?
2. Cuantas vacunas se aplicaron discriminadas por laboratorio por cada region?
3. Existe relacion en el tipo de vacuna mas aplicada en cada region y cantidad de muertes.

In [ ]:
# Agrupar Muertes por Region (Departamento)
muertes_por_region = Muertes.groupby('Nombre departamento').size().reset_index(name='Muertes')

# Por comodidad vamos a trabajar con el top 10 de regiones con mayor numero de muertes
top_regiones = muertes_por_region.nlargest(10, 'Muertes')


print(top_regiones)

En esta parte se analizan los datos contentidos en el Dataframe Vacunas y se observa que algunas para el mismo labortatorio de vacunas el archivo las reporta con diferentes formas de escritura como como en el caso de Astrazeneca. Mismo escenario para los territorios.

Se normaliza la data y se filtra para tener una tabla con mejor visibilidad y que contenga solo la informacion deseable para el analisis.

Para normalizar la data se crean dos diccionarions que reemplazaran la informacion contenida en el dataset para grupar los laboratorios y los territorios.

In [ ]:
# Normalizacion de las variables

labs_corrections = {
    'ASTRAZENECA': 'AZTRAZENECA',
    'astrezenca': 'AZTRAZENECA',  
    'pfizer': 'PFIZER',          
    'sinovac ': 'SINOVAC'         
}
Vacunas['Laboratorio_Vacuna'] = Vacunas['Laboratorio_Vacuna'].replace(labs_corrections)

territorio_corrections = {
    "ATLÁNTICO" : "ATLANTICO",
    'ATLANTICO' : 'ATLANTICO',
    'SANTA MARTHA' : 'SANTA MARTA',
    'SANTA MARTA' : 'SANTA MARTA',
    'BOGOTÁ D.C.' : 'BOGOTA D.C.',
    'BOGOTA D.C.' : 'BOGOTA D.C.',
    'BOYACÁ' : 'BOYACA',
    'BOYACA' : 'BOYACA',
    'CAQUETÁ' : 'CAQUETA',
    'CAQUETA' : 'CAQUETA',
    'CHOCÓ' : 'CHOCO',
    'CHOCO' : 'CHOCO',
    'CÓRDOBA' : 'CORDOBA',
    'CORDOBA' : 'CORDOBA',
    'LA_GUAJIRA' : 'LA GUAJIRA',
    'LA GUAJIRA' : 'LA GUAJIRA',
    'QUINDÍO' : 'QUINDIO',
    'QUINDIO' : 'QUINDIO',
    'VALLE_DEL_CAUCA' : 'VALLE DEL CAUCA',
    'VALLE DEL CAUCA' : 'VALLE DEL CAUCA',
    'VAUPÉS' :  'VAUPES',
    'VAUPES' :  'VAUPES',
    'SAN ANDRES, PROVIDENCIA Y SANTA CATALINA' :'SAN ANDRES',
    'BOLÍVAR' : 'BOLIVAR',
    'BOLIVAR': 'BOLIVAR'
}

Vacunas['Nom_Territorio'] = Vacunas['Nom_Territorio'].replace(territorio_corrections)

# Filtrado de Datos 

vacunas_filtradas = Vacunas[['Nom_Territorio', 'Laboratorio_Vacuna', 'Cantidad']].dropna()

vacunas_pivot = vacunas_filtradas.pivot_table(
    index='Nom_Territorio',
    columns='Laboratorio_Vacuna',
    values='Cantidad',
    aggfunc='sum',
    fill_value=0
).reset_index()

vacunas_pivot.columns.name = None  

print(vacunas_pivot)